# Tarjetas Black


Vamos a utilizar un dataset de movimientos de tarjetas de crédito que provienen de tarjetas Black. 

El origen es el siguiente repositorio de GitHub (aunque han sido preprocesados en R):

https://github.com/splatsh/tarjetasblack

Originalmente la información viene de la siguiente fuente:

https://www.cuartopoder.es/multimedia/2014/10/11/gastos-de-los-exdirectivos-de-caja-madrid-uno-a-uno-con-las-tarjetas-negras-tabla/3403


## Modelo conceptual

<br><br> 

<img src="./images/Modelo_conceptual.png" alt="Example">
<br><br> 

## Modelo de relación

<br><br> 

<img src="./images/Modelo_relacion.png" alt="Example" width="500" height="500">

<br><br> 

## Contenido de la información

Contiene la siguiente información:

- Nombre: Nombre de la persona que posee la tarjeta y realiza el movimiento
- Fecha: Fecha del movimiento
- Hora: Hora del movimiento
- Minuto: Minuto del movimiento
- Importe: Importe del movimiento
- Comercio donde se realiza el movimiento
- Actividad Completa: Detalle de la que se dedica el comercio
- Actividad: Contiene una clasificación de la actividad del comercio. Una Actividad puede agrupar a varios comercios
- Función: Función que desempeña la persona dentro de su organización (Sólo para los partidos políticos)
- Organización: Organización a la que pertenece la persona que realiza el gasto

Podrían existir movimientos con campos vacíos.

## Formato del fichero

El fichero con los datos está comprimido, en formato CSV, y utiliza el carácter ~ cómo separador de columnas.


## Preguntas

Vamos a intentar resolver las siguientes preguntas:
    
- Los 10 movimientos mas caros por actividad
- Los 10 movimientos mas caros
- Los movimientos de una persona concreta (ordenados por importe)
- Las 10 personas que mas han gastado
- Importes de una persona agrupados por actividad
- ¿Quién se gasta más, los concejales o los directivos?
- ¿Qué organización se gasta más?
- ¿Qué comercio es más popular entre los miembros?

# Preprocesado de datos

Se realizará un preprocesado de datos cuyo objetico es limpiar y normalizar los datos para que se puedan utilizar más fácilmente por el resto de prácticas  

El formato de salida será un fichero Microsoft Excel que contendrá los siguientes datasets:

- Movimientos: Movimientos donde se han eliminado los atributos personales y se han sustituido por un ID
- Miembros: Dataset con todas las personas que han realzado movimientos

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../../data/black.txt.gz", compression='gzip', sep='~',  index_col=False, low_memory=False)

In [3]:
df.head()

,nombre,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,funcion,organizacion
0,Alberto Recarte García Andrade,2003-01-03,12,30,38.70,RCG OFICINA,CONFECCION TEXTIL EN GENERAL,ROPA,concejal,Partido Popular
1,Alberto Recarte García Andrade,2003-01-03,12,32,14.60,MANZANIL AREA,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",HOTEL,concejal,Partido Popular
2,Alberto Recarte García Andrade,2003-01-04,19,7,95.62,REST REAL C GOLF SOTOGRAN,RESTAURANTES RESTO,RESTAURANTE,concejal,Partido Popular
3,Alberto Recarte García Andrade,2003-01-07,15,31,49.13,ESTACONES DE SERVICIO ML,GASOLINERAS,COCHE,concejal,Partido Popular
4,Alberto Recarte García Andrade,2003-01-07,16,5,13.93,NISA (H 24) VINHOLAN S.A.,GASOLINERAS,COCHE,concejal,Partido Popular


Nos quedamos sólo con los registros únicos

In [4]:
df_miembros = df.filter(["nombre", "funcion", "organizacion"]) \
                .drop_duplicates()
df_miembros.head()

,nombre,funcion,organizacion
0,Alberto Recarte García Andrade,concejal,Partido Popular
1305,Alejandro Couceiro Ojeda,concejal,CEIM
1851,Ángel Eugenio Gómez del Pulgar Perales,concejal,PSOE
2832,Angel Rizaldos González,concejal,Izquierda Unida
2952,Antonio Cámara Eguinoa,concejal,Partido Popular


Se crea un dataset de miembros, asignado a cada persona un ID único

In [5]:
df_miembros.sort_values(by = "nombre").assign(
        id_miembro = df_miembros.nombre.rank().astype(int)
).head()

,nombre,funcion,organizacion,id_miembro
0,Alberto Recarte García Andrade,concejal,Partido Popular,1
1305,Alejandro Couceiro Ojeda,concejal,CEIM,2
2832,Angel Rizaldos González,concejal,Izquierda Unida,3
2952,Antonio Cámara Eguinoa,concejal,Partido Popular,4
6292,Antonio Rey de Viñas Sánchez-Majestad,concejal,CC OO,5


In [7]:
df_miembros = (df_miembros
    .assign(id_miembro = df_miembros.nombre.rank(method='min').astype(int))
    .filter(['id_miembro', 'nombre', 'funcion', 'organizacion'])
)               

In [8]:
df_miembros.head()

,id_miembro,nombre,funcion,organizacion
0,1,Alberto Recarte García Andrade,concejal,Partido Popular
1305,2,Alejandro Couceiro Ojeda,concejal,CEIM
1851,83,Ángel Eugenio Gómez del Pulgar Perales,concejal,PSOE
2832,3,Angel Rizaldos González,concejal,Izquierda Unida
2952,4,Antonio Cámara Eguinoa,concejal,Partido Popular


Se inserta en el dataset principal el ID creado anteriormente

In [9]:
df_result = pd.merge(df, df_miembros, on = ['nombre', 'funcion', 'organizacion'], how = 'inner')
df_result.head()

,nombre,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,funcion,organizacion,id_miembro
0,Alberto Recarte García Andrade,2003-01-03,12,30,38.70,RCG OFICINA,CONFECCION TEXTIL EN GENERAL,ROPA,concejal,Partido Popular,1
1,Alberto Recarte García Andrade,2003-01-03,12,32,14.60,MANZANIL AREA,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",HOTEL,concejal,Partido Popular,1
2,Alberto Recarte García Andrade,2003-01-04,19,7,95.62,REST REAL C GOLF SOTOGRAN,RESTAURANTES RESTO,RESTAURANTE,concejal,Partido Popular,1
3,Alberto Recarte García Andrade,2003-01-07,15,31,49.13,ESTACONES DE SERVICIO ML,GASOLINERAS,COCHE,concejal,Partido Popular,1
4,Alberto Recarte García Andrade,2003-01-07,16,5,13.93,NISA (H 24) VINHOLAN S.A.,GASOLINERAS,COCHE,concejal,Partido Popular,1


Seleccionamos las columnas que nos interesan

In [10]:
df_movimientos = df_result.filter(
    ['id_miembro','fecha', 'hora', 'minuto', 'importe', 'comercio', 'actividad_completa', 'actividad']
)

Eliminamos la información horaria de la fecha

In [11]:
df_movimientos = df_movimientos.assign(
    fecha = pd.to_datetime(df['fecha']).dt.date
)
df_movimientos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76238 entries, 0 to 76237
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_miembro          76238 non-null  int64  
 1   fecha               76238 non-null  object 
 2   hora                76238 non-null  int64  
 3   minuto              76238 non-null  int64  
 4   importe             76238 non-null  float64
 5   comercio            68500 non-null  object 
 6   actividad_completa  76221 non-null  object 
 7   actividad           76220 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 5.2+ MB


In [12]:
df_movimientos.head()

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad
0,1,2003-01-03,12,30,38.70,RCG OFICINA,CONFECCION TEXTIL EN GENERAL,ROPA
1,1,2003-01-03,12,32,14.60,MANZANIL AREA,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",HOTEL
2,1,2003-01-04,19,7,95.62,REST REAL C GOLF SOTOGRAN,RESTAURANTES RESTO,RESTAURANTE
3,1,2003-01-07,15,31,49.13,ESTACONES DE SERVICIO ML,GASOLINERAS,COCHE
4,1,2003-01-07,16,5,13.93,NISA (H 24) VINHOLAN S.A.,GASOLINERAS,COCHE


Y por último selecionamos al azar un 10% de los datos para que la práctica sea más ágil ...

In [13]:
df_movimientos_lite = df_movimientos.sample(frac=0.10, random_state = 1313)
df_movimientos_lite.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 51501 to 35498
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_miembro          7624 non-null   int64  
 1   fecha               7624 non-null   object 
 2   hora                7624 non-null   int64  
 3   minuto              7624 non-null   int64  
 4   importe             7624 non-null   float64
 5   comercio            6882 non-null   object 
 6   actividad_completa  7621 non-null   object 
 7   actividad           7621 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 536.1+ KB


In [14]:
df_movimientos_lite.head()

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad
51501,77,2006-04-17,11,40,44.56,FCIA TORRES MU¬OZ,FARMACIAS,SALUD
13619,15,2010-09-11,14,26,67.56,E.S.VEGONES DER,C.A.M.P.S.A.,COCHE
22298,29,2003-01-24,14,15,191.00,EL CORTE INGLES,EL CORTE INGLES,COMPRA BIENES
65305,74,2007-05-10,15,2,599.20,VIAJES ECI,EL CORTE INGLES,COMPRA BIENES
52699,77,2010-12-18,14,35,3.65,PK VELAZQUEZ JORGE JUAN,GARAJES Y APARCAMIENTOS,COCHE


Grabamos el resultado en un fichero Excel donde cada dataset estará en un hoja diferente

In [15]:
writer = pd.ExcelWriter("../../data/black.xlsx")
df_movimientos_lite.to_excel(writer,'Movimientos', index=False)
df_miembros.to_excel(writer, 'Miembros', index=False)
writer.save()